In [4]:
import subprocess, sys, json, csv, os, re
from datetime import datetime
from pathlib import Path

In [6]:
def run_js_generator(json_path: str) -> dict:
    """Run the Node.js generator and parse its JSON output."""
    js_script = "generate_resume.js"
    result = subprocess.run(
        ["node", str(js_script), json_path],
        capture_output=True, text=True, encoding="utf-8"
    )

    if result.returncode != 0:
        print("JS Generator stderr:", result.stderr)
        sys.exit(1)

    # The JS script prints status lines and then a JSON object on the last line
    lines = result.stdout.strip().split("\n")
    for line in lines:
        if not line.startswith("{"):
            print(line)  # Print status messages (e.g. "✓ DOCX written: ...")

    # Parse the JSON output (last line)
    try:
        output_info = json.loads(lines[-1])
    except json.JSONDecodeError:
        print("Error: Could not parse JS generator output.")
        print("Full stdout:", result.stdout)
        sys.exit(1)

    return output_info

from docx2pdf import convert

def convert_to_pdf(docx_path):
    """Convert DOCX to PDF using docx2pdf. PDFs go to output/, not output/docx files/."""
    pdf_path = str(Path(docx_path).parent.parent / Path(docx_path).stem) + ".pdf"
    convert(docx_path, pdf_path, keep_active=True)
    print(f"✓ PDF written: {pdf_path}")
    return pdf_path
    

def update_tracker(data: dict, docx_path: str, pdf_path: str):
    """Append a row to tracker.csv."""
    tracker_path = Path("tracker.csv")

    # Read existing content to check if header exists
    file_exists = tracker_path.exists() and tracker_path.stat().st_size > 0

    row = {
        "date": data["metadata"].get("date_applied", datetime.now().strftime("%Y-%m-%d")),
        "company": data["metadata"].get("target_company", ""),
        "role": data["metadata"].get("target_role", ""),
        "status": data["metadata"].get("status", "draft"),
        "resume_file": os.path.basename(docx_path),
        "cover_letter_file": "yes" if (data.get("cover_letter") and data["cover_letter"].get("opening")) else "no",
        "notes": data["metadata"].get("notes", "")
    }

    with open(tracker_path, "a", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=row.keys())
        if not file_exists:
            writer.writeheader()
        writer.writerow(row)

    print(f"✓ Tracker updated: {tracker_path}")

def expected_output_exists(data: dict) -> bool:
    """Mirror the JS filename logic to check if this application has already been generated."""
    name = data["header"]["name"].split(",")[0].strip()
    company = re.sub(r'[^a-zA-Z0-9 ]', '', data["metadata"]["target_company"]).strip()
    role = re.sub(r'[^a-zA-Z0-9 ]', '', data["metadata"]["target_role"]).strip()
    base = f"{name}_{company}_{role}"
    return Path("output", "docx files", f"{base}_resume.docx").exists()


def generate_all():
    app_dir = Path("applications")
    json_files = sorted(app_dir.glob("*.json"))

    if not json_files:
        print("No JSON files found in applications/")
        return

    pending, skipped = [], []

    for json_path in json_files:
        with open(json_path) as f:
            data = json.load(f)

        if expected_output_exists(data):
            skipped.append(json_path.name)
            continue

        pending.append((json_path, data))

    print(f"\n📊 Found {len(json_files)} application(s): {len(pending)} to generate, {len(skipped)} already done.\n")

    if skipped:
        print(f"  ⏭️  Skipping: {', '.join(skipped)}\n")

    for json_path, data in pending:
        print(f"📄 Generating: {json_path.name}")

        output_info = run_js_generator(str(json_path))
        docx_path = output_info["docx"]
        pdf_path = convert_to_pdf(docx_path)

        if output_info.get("coverLetterDocx"):
            convert_to_pdf(output_info["coverLetterDocx"])

        update_tracker(data, docx_path, pdf_path)
        print(f"  ✅ Done.\n")

    print(f"✅ All {len(pending)} resume(s) generated.\n")

C:\Users\matto\anaconda3\envs\words\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
generate_all()


📊 Found 43 application(s): 3 to generate, 40 already done.

  ⏭️  Skipping: 2026-02-01_1password_vp-data-analytics.json, 2026-02-01_actblue_director-engineering-data-science.json, 2026-02-01_apogee-therapeutics_director-data-engineering-analytics.json, 2026-02-01_hello-heart_director-analytics.json, 2026-02-01_incyte_machine-learning-engineer.json, 2026-02-01_supercom_director-analytics-data-science.json, 2026-02-01_symetra_avp-data-analytics.json, 2026-02-01_test-company_senior-data-scientist.json, 2026-02-02_alimentiv_director-enterprise-data.json, 2026-02-02_amgen_responsible-ai-manager.json, 2026-02-02_biogen_director-advanced-analytics-data-science-ai.json, 2026-02-02_danaher_senior-director-ai-data-infrastructure.json, 2026-02-02_insulet_director-data-biostatistics.json, 2026-02-02_natera_director-data-ai-governance.json, 2026-02-02_veeva-systems_senior-director-engineering-data-operations.json, 2026-02-03_bridgeview_director-data-analytics.json, 2026-02-03_mckesson_director-dat

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.89s/it]


✓ PDF written: C:\Users\matto\OneDrive\Python\resume-generator\output\Matt Oremland_Alignment Health_Director of Data Science_resume.pdf


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.31it/s]


✓ PDF written: C:\Users\matto\OneDrive\Python\resume-generator\output\Matt Oremland_Alignment Health_Director of Data Science_cover_letter.pdf
✓ Tracker updated: tracker.csv
  ✅ Done.

📄 Generating: 2026-02-23_dynatron_sr-director-data.json
✓ DOCX written: C:\Users\matto\OneDrive\Python\resume-generator\output\docx files\Matt Oremland_Dynatron_Sr Director of Data_resume.docx
✓ Cover letter DOCX written: C:\Users\matto\OneDrive\Python\resume-generator\output\docx files\Matt Oremland_Dynatron_Sr Director of Data_cover_letter.docx


...

✓ PDF written: C:\Users\matto\OneDrive\Python\resume-generator\output\Matt Oremland_Dynatron_Sr Director of Data_cover_letter.pdf
✓ Tracker updated: tracker.csv
  ✅ Done.

📄 Generating: 2026-02-23_llr-partners_director-ai-strategy-value-creation.json
✓ DOCX written: C:\Users\matto\OneDrive\Python\resume-generator\output\docx files\Matt Oremland_LLR Partners_Director of AI Strategy  Value Creation_resume.docx
✓ Cover letter DOCX written: C:\Users\matto\OneDrive\Python\resume-generator\output\docx files\Matt Oremland_LLR Partners_Director of AI Strategy  Value Creation_cover_letter.docx


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.76s/it]


✓ PDF written: C:\Users\matto\OneDrive\Python\resume-generator\output\Matt Oremland_LLR Partners_Director of AI Strategy  Value Creation_resume.pdf


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.62s/it]

✓ PDF written: C:\Users\matto\OneDrive\Python\resume-generator\output\Matt Oremland_LLR Partners_Director of AI Strategy  Value Creation_cover_letter.pdf
✓ Tracker updated: tracker.csv
  ✅ Done.

✅ All 3 resume(s) generated.



In [56]:
INPUT_JSON = "2026-02-01_test-company_senior-data-scientist.json"

def generate(json_file):
    json_path = os.path.join('applications', json_file)
    
    if not Path(json_path).exists():
        print(f"Error: File not found: {json_path}")
        return
    
    print(f"\n📄 Generating resume from: {json_path}\n")
    
    with open(json_path) as f:
        data = json.load(f)
    
    # Step 1: Generate DOCX(s) via JS
    output_info = run_js_generator(json_path)
    docx_path = output_info["docx"]
    
    # Step 2: Convert resume to PDF
    pdf_path = convert_to_pdf(docx_path)
    
    # Step 3: Convert cover letter to PDF if it exists
    if output_info.get("coverLetterDocx"):
        convert_to_pdf(output_info["coverLetterDocx"])
    
    # Step 4: Update tracker
    update_tracker(data, docx_path, pdf_path)
    
    print(f"\n✅ Done! Files are in: output/\n")

generate(INPUT_JSON)


📄 Generating resume from: applications\2026-02-01_test-company_senior-data-scientist.json

✓ DOCX written: C:\Users\matto\OneDrive\Python\resume-generator\output\docx files\Matt Oremland_Test Company_Senior Data Scientist_resume.docx
✓ Cover letter DOCX written: C:\Users\matto\OneDrive\Python\resume-generator\output\docx files\Matt Oremland_Test Company_Senior Data Scientist_cover_letter.docx



%|                                                                                            | 0/1 [00:00<?, ?it/s]
100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.77s/it]


✓ PDF written: C:\Users\matto\OneDrive\Python\resume-generator\output\Matt Oremland_Test Company_Senior Data Scientist_resume.pdf



%|                                                                                            | 0/1 [00:00<?, ?it/s]
100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.28s/it]

✓ PDF written: C:\Users\matto\OneDrive\Python\resume-generator\output\Matt Oremland_Test Company_Senior Data Scientist_cover_letter.pdf
✓ Tracker updated: tracker.csv

✅ Done! Files are in: output/

